# MP2 二阶梯度：安全做法——结合轨道旋转的微分 Z-Vector 方程

这篇文档将会使用安全的做法 (轨道旋转得到的全轨道 U 矩阵)，以及计算量和储存量较小的微分 Z-Vector 方程与逆向 Z-Vector 方程的方法，解决 MP2 的二阶梯度．

这篇文档将不会详细描述计算细节．代码类似于 [微分 Z-Vector 方程文档](mp2_hess_deriv_zvector.ipynb)，但采用了轨道旋转的 U 矩阵．

In [1]:
import numpy as np
from pyscf import scf, gto, lib, grad, hessian, dft, mp
import pyscf.hessian.rks
import pyscf.grad.rks
from functools import partial
import pickle
import matplotlib.pyplot as plt

from utilities import val_from_fchk, NumericDiff
from hessian import HFHelper, GGAHelper

np.einsum = partial(np.einsum, optimize=["greedy", 1024 ** 3 * 2 / 8])
np.einsum_path = partial(np.einsum_path, optimize=["greedy", 1024 ** 3 * 2 / 8])
np.set_printoptions(6, linewidth=150, suppress=True)

In [2]:
mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.5  0.0  0.0
H  0.0  0.7  1.5
"""
mol.basis = "6-31G"
mol.verbose = 0
mol.build()

nmo = nao = mol.nao
natm = mol.natm
nocc = mol.nelec[0]
nvir = nmo - nocc
so = slice(0, nocc)
sv = slice(nocc, nmo)
sa = slice(0, nmo)

### 文档定义所用变量

In [3]:
# We only U_1 in this document
# Pre-generate some code here will make later work quicker
# But these operations or memory consuming
hfh = HFHelper(mol)
hfh.U_1_vo
hfh.eri0_mo
hfh.eri1_mo
hfh.eri2_mo
hfh

/share/home/zyzhu/Git-Repo/Python-xDH/pyxdh/hessian/hf_helper.py:229: UserWarning: eri0_mo: ERI AO -> MO is quite expensive!
  warnings.warn("eri0_mo: ERI AO -> MO is quite expensive!")
/share/home/zyzhu/Git-Repo/Python-xDH/pyxdh/hessian/hf_helper.py:222: UserWarning: eri0_ao: ERI should not be stored in memory! Consider J/K engines!
  warnings.warn("eri0_ao: ERI should not be stored in memory! Consider J/K engines!")
/share/home/zyzhu/Git-Repo/Python-xDH/pyxdh/hessian/hf_helper.py:279: UserWarning: eri1_mo: 4-idx tensor ERI should be not used!
  warnings.warn("eri1_mo: 4-idx tensor ERI should be not used!")
/share/home/zyzhu/Git-Repo/Python-xDH/pyxdh/hessian/hf_helper.py:272: UserWarning: eri1_ao: 4-idx tensor ERI should be not used!
  warnings.warn("eri1_ao: 4-idx tensor ERI should be not used!")
/share/home/zyzhu/Git-Repo/Python-xDH/pyxdh/hessian/hf_helper.py:317: UserWarning: eri2_mo: 4-idx tensor ERI should be not used!
  warnings.warn("eri2_mo: 4-idx tensor ERI should be not used

In [4]:
%%capture

e, eo, ev = hfh.e, hfh.eo, hfh.ev
C, Co, Cv = hfh.C, hfh.Co, hfh.Cv
D = hfh.D
eri0_mo = hfh.eri0_mo
eri1_mo = hfh.eri1_mo
eri2_mo = hfh.eri2_mo

F_1_mo = hfh.F_1_mo
S_1_mo = hfh.S_1_mo
F_2_mo = hfh.F_2_mo
S_2_mo = hfh.S_2_mo
U_1_vo = hfh.U_1_vo
U_1_ov = hfh.U_1_ov
Ax0_Core = hfh.Ax0_Core
Ax1_Core = hfh.Ax1_Core

D_iajb = lib.direct_sum("i - a + j - b", hfh.eo, hfh.ev, hfh.eo, hfh.ev)

MP2 Hessian 参考值如下：

In [5]:
g_array = val_from_fchk("Cartesian Force Constants", "include/mp2_hess/mp2_hess.fchk")
d_hess = natm * 3
hess_mp2_gaussian = np.zeros((d_hess, d_hess))
p = 0
for d1 in range(d_hess):
    for d2 in range(d1 + 1):
        hess_mp2_gaussian[d1][d2] = hess_mp2_gaussian[d2][d1] = g_array[p]
        p += 1
hess_mp2_gaussian = hess_mp2_gaussian.reshape((natm, 3, natm, 3)).swapaxes(1, 2)
hess_mp2_ref = hess_mp2_gaussian - hfh.scf_hess.kernel()

## 必要变量定义

### MP2 相关能计算必要量

In [6]:
mp2_eng = mp.MP2(hfh.scf_eng)
mp2_eng.kernel()[0]
mp2_grad = grad.mp2.Gradients(mp2_eng)
mp2_grad.kernel()
grad_mp2_ref = mp2_grad.de - hfh.scf_grad.kernel()

In [7]:
g_mo = eri0_mo
G_mo = 2 * g_mo - g_mo.swapaxes(-1, -3)
g_iajb = g_mo[so, sv, so, sv]
G_iajb = G_mo[so, sv, so, sv]
D_iajb = lib.direct_sum("i - a + j - b", eo, ev, eo, ev)
t_iajb = g_iajb / D_iajb
T_iajb = 2 * t_iajb - t_iajb.swapaxes(-1, -3)

In [8]:
np.allclose((g_iajb * G_iajb / D_iajb).sum(), mp2_eng.e_corr)

True

### 旋转下 U 矩阵的定义

In [9]:
U_1R = - 0.5 * S_1_mo
U_1R[:, :, sv, so] = U_1_vo
U_1R[:, :, so, sv] = - S_1_mo[:, :, so, sv] - U_1_vo.swapaxes(-1, -2)

In [10]:
F_1R = (
    + F_1_mo
    + np.einsum("Atpq, p -> Atpq", U_1R, e)
    + np.einsum("Atqp, q -> Atpq", U_1R, e)
    + Ax0_Core(sa, sa, sv, so)(U_1_vo)
    - 0.5 * Ax0_Core(sa, sa, so, so)(S_1_mo[:, :, so, so])
)

### MP2 梯度计算必要量

In [11]:
pd_g_mo = eri1_mo
pd_g_iajb = pd_g_mo[:, :, so, sv, so, sv]

In [12]:
D_r = np.zeros((nmo, nmo))
D_r[so, so] += - 2 * np.einsum("iakb, jakb -> ij", T_iajb, t_iajb)
D_r[sv, sv] += 2 * np.einsum("iajc, ibjc -> ab", T_iajb, t_iajb)

L = np.zeros((nvir, nocc))
L += Ax0_Core(sv, so, sa, sa)(D_r)
L -= 4 * np.einsum("jakb, ijbk -> ai", T_iajb, g_mo[so, so, sv, so])
L += 4 * np.einsum("ibjc, abjc -> ai", T_iajb, g_mo[sv, sv, so, sv])

D_r[sv, so] = scf.cphf.solve(Ax0_Core(sv, so, sv, so), e, hfh.mo_occ, L, max_cycle=100, tol=1e-13)[0]

In [13]:
# W[I] - Correct with s1-im1 term in PySCF
D_WI = np.zeros((nmo, nmo))
D_WI[so, so] = - 2 * np.einsum("iakb, jakb -> ij", T_iajb, g_mo[so, sv, so, sv])
D_WI[sv, sv] = - 2 * np.einsum("iajc, ibjc -> ab", T_iajb, g_mo[so, sv, so, sv])
D_WI[sv, so] = - 4 * np.einsum("jakb, ijbk -> ai", T_iajb, g_mo[so, so, sv, so])

# W[II] - Correct with s1-zeta term in PySCF
# Note that zeta in PySCF includes HF energy weighted density rdm1e
# The need of scaler 1 in D_WII[sv, so] is that Aikens use doubled P
D_WII = np.zeros((nmo, nmo))
D_WII[so, so] = - 0.5 * D_r[so, so] * lib.direct_sum("i + j -> ij", eo, eo)
D_WII[sv, sv] = - 0.5 * D_r[sv, sv] * lib.direct_sum("a + b -> ab", ev, ev)
D_WII[sv, so] = - D_r[sv, so] * eo

# W[III] - Correct with s1-vhf_s1occ term in PySCF
D_WIII = np.zeros((nmo, nmo))
D_WIII[so, so] = - 0.5 * Ax0_Core(so, so, sa, sa)(D_r)

# Summation
D_W = D_WI + D_WII + D_WIII

In [14]:
np.allclose(
    + (D_r * F_1_mo).sum(axis=(-1, -2))
    + (D_W * S_1_mo).sum(axis=(-1, -2))
    + (2 * T_iajb * pd_g_iajb).sum(axis=(-1, -2, -3, -4)),
    mp2_grad.de - hfh.scf_grad.de
)

True

## MP2 二阶梯度：轨道旋转的 Z-Vector 方程解法

### 第一贡献项 $D_{pq}^\mathrm{MP2} (\frac{\partial}{\partial B_s} F_{pq}^{A_t})$

In [15]:
pdR_F_1_mo = (
    + hfh.F_2_mo
    + np.einsum("Atpm, Bsmq -> ABtspq", hfh.F_1_mo, U_1R)
    + np.einsum("Atmq, Bsmp -> ABtspq", hfh.F_1_mo, U_1R)
    + hfh.Ax1_Core(sa, sa, sa, so)(U_1R[:, :, :, so])
)

In [16]:
hess_mp2_contrib1 = np.einsum("pq, ABtspq -> ABts", D_r, pdR_F_1_mo)

### 第二贡献项 $W_{pq}^\mathrm{MP2} (\frac{\partial}{\partial B_s} S_{pq}^{A_t})$

这一项的难点也仅仅在于求取 `pdA_S_1_mo` $(\frac{\partial}{\partial B_s} S_{pq}^{A_t})$．

In [17]:
pdR_S_1_mo = (
    + hfh.S_2_mo
    + np.einsum("Atpm, Bsmq -> ABtspq", hfh.S_1_mo, U_1R)
    + np.einsum("Atmq, Bsmp -> ABtspq", hfh.S_1_mo, U_1R)
)

In [18]:
hess_mp2_contrib2 = np.einsum("pq, ABtspq -> ABts", D_W, pdR_S_1_mo)

### 第五、六贡献项 $2 (\frac{\partial}{\partial B_s} T_{ij}^{ab}) (\partial_{A_t} g_{ij}^{ab}) + 2 T_{ij}^{ab} (\frac{\partial}{\partial B_s} \partial_{A_t} g_{ij}^{ab})$

In [19]:
pd_g_mo = eri1_mo
pdRU_g_mo = (
    + np.einsum("pjkl, Atpi -> Atijkl", g_mo, U_1R)
    + np.einsum("ipkl, Atpj -> Atijkl", g_mo, U_1R)
    + np.einsum("ijpl, Atpk -> Atijkl", g_mo, U_1R)
    + np.einsum("ijkp, Atpl -> Atijkl", g_mo, U_1R)
)
pdR_g_mo = pd_g_mo + pdRU_g_mo
pdR_G_mo = 2 * pdR_g_mo - pdR_g_mo.swapaxes(-1, -3)

pdR_g_iajb = pdR_g_mo[:, :, so, sv, so, sv]
pdR_G_iajb = pdR_G_mo[:, :, so, sv, so, sv]

In [20]:
pd_pd_g_mo = eri2_mo
pdRU_pd_g_mo = (
    # pd on g, U matrix on B
    + np.einsum("Atpjkl, Bspi -> ABtsijkl", pd_g_mo, U_1R)
    + np.einsum("Atipkl, Bspj -> ABtsijkl", pd_g_mo, U_1R)
    + np.einsum("Atijpl, Bspk -> ABtsijkl", pd_g_mo, U_1R)
    + np.einsum("Atijkp, Bspl -> ABtsijkl", pd_g_mo, U_1R)
)
pdR_pd_g_mo = pd_pd_g_mo + pdRU_pd_g_mo
pdR_pd_g_iajb = pdR_pd_g_mo[:, :, :, :, so, sv, so, sv]

In [21]:
pdR_t_iajb = (
    + pdR_g_iajb / D_iajb
    - np.einsum("Atki, kajb, iajb -> Atiajb", F_1R[:, :, so, so], t_iajb, 1 / D_iajb)
    - np.einsum("Atkj, iakb, iajb -> Atiajb", F_1R[:, :, so, so], t_iajb, 1 / D_iajb)
    + np.einsum("Atca, icjb, iajb -> Atiajb", F_1R[:, :, sv, sv], t_iajb, 1 / D_iajb)
    + np.einsum("Atcb, iajc, iajb -> Atiajb", F_1R[:, :, sv, sv], t_iajb, 1 / D_iajb)
)
pdR_T_iajb = 2 * pdR_t_iajb - pdR_t_iajb.swapaxes(-1, -3)

In [22]:
(
    + np.einsum("Atiajb, iajb -> At", pdR_T_iajb, g_iajb)
    + np.einsum("iajb, Atiajb -> At", T_iajb, pdR_g_iajb)
) - (mp2_grad.de - hfh.scf_grad.de)

array([[-0., -0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0., -0.],
       [-0., -0., -0.]])

In [23]:
hess_mp2_contrib5_6 = (
    + 2 * np.einsum("Bsiajb, Atiajb -> ABts", pdR_T_iajb, pd_g_iajb)
    + 2 * np.einsum("iajb, ABtsiajb -> ABts", T_iajb, pdR_pd_g_iajb)
)

### 第三、四贡献项 $(\frac{\partial}{\partial B_s} D_{pq}^\mathrm{MP2}) F_{pq}^{A_t} + (\frac{\partial}{\partial B_s} W_{pq}^\mathrm{MP2}) S_{pq}^{A_t}$

In [24]:
pdR_D_r = np.zeros((natm, 3, nmo, nmo))
pdR_D_r[:, :, so, so] -= 2 * np.einsum("iakb, Atjakb -> Atij", T_iajb, pdR_t_iajb)
pdR_D_r[:, :, sv, sv] += 2 * np.einsum("iajc, Atibjc -> Atab", T_iajb, pdR_t_iajb)
pdR_D_r[:, :, so, so] -= 2 * np.einsum("Atiakb, jakb -> Atij", pdR_T_iajb, t_iajb)
pdR_D_r[:, :, sv, sv] += 2 * np.einsum("Atiajc, ibjc -> Atab", pdR_T_iajb, t_iajb)

RHS = np.zeros((natm, 3, nvir, nocc))
# D_r Part
RHS += hfh.Ax0_Core(sv, so, sa, sa)(pdR_D_r)
RHS += hfh.Ax1_Core(sv, so, sa, sa)(np.array([[D_r]]))[:, 0, :, 0]
RHS += np.einsum("Atpa, pi -> Atai", U_1R[:, :, :, sv], hfh.Ax0_Core(sa, so, sa, sa)(D_r))
RHS += np.einsum("Atpi, ap -> Atai", U_1R[:, :, :, so], hfh.Ax0_Core(sv, sa, sa, sa)(D_r))
RHS += Ax0_Core(sv, so, sa, sa)(np.einsum("Atmp, pq -> Atmq", U_1R, D_r))
RHS += Ax0_Core(sv, so, sa, sa)(np.einsum("Atmq, pq -> Atpm", U_1R, D_r))
# (ea - ei) * Dai
RHS += np.einsum("Atca, ci -> Atai", F_1R[:, :, sv, sv], D_r[sv, so])
RHS -= np.einsum("Atki, ak -> Atai", F_1R[:, :, so, so], D_r[sv, so])
# 2-pdm part
RHS -= 4 * np.einsum("Atjakb, ijbk -> Atai", pdR_T_iajb, g_mo[so, so, sv, so])
RHS += 4 * np.einsum("Atibjc, abjc -> Atai", pdR_T_iajb, g_mo[sv, sv, so, sv])
RHS -= 4 * np.einsum("jakb, Atijbk -> Atai", T_iajb, pdR_g_mo[:, :, so, so, sv, so])
RHS += 4 * np.einsum("ibjc, Atabjc -> Atai", T_iajb, pdR_g_mo[:, :, sv, sv, so, sv])

In [25]:
# o-o and v-v part contribution of original contrib3
# code is the same, however result value is different compared with contrib3
hess_mp2_contrib3_4 = np.einsum("Bspq, Atpq -> ABts", pdR_D_r, F_1_mo)

In [26]:
# WI part is the same as before
pdR_D_W = np.zeros((natm, 3, nmo, nmo))
pdR_D_W[:, :, so, so] -= 2 * np.einsum("Atiakb, jakb -> Atij", pdR_T_iajb, g_mo[so, sv, so, sv])
pdR_D_W[:, :, sv, sv] -= 2 * np.einsum("Atiajc, ibjc -> Atab", pdR_T_iajb, g_mo[so, sv, so, sv])
pdR_D_W[:, :, sv, so] -= 4 * np.einsum("Atjakb, ijbk -> Atai", pdR_T_iajb, g_mo[so, so, sv, so])
pdR_D_W[:, :, so, so] -= 2 * np.einsum("iakb, Atjakb -> Atij", T_iajb, pdR_g_mo[:, :, so, sv, so, sv])
pdR_D_W[:, :, sv, sv] -= 2 * np.einsum("iajc, Atibjc -> Atab", T_iajb, pdR_g_mo[:, :, so, sv, so, sv])
pdR_D_W[:, :, sv, so] -= 4 * np.einsum("jakb, Atijbk -> Atai", T_iajb, pdR_g_mo[:, :, so, so, sv, so])

# WII's pdR_D_r[:, :, sv, so] contribution is zero
pdR_D_W[:, :, so, so] -= pdR_D_r[:, :, so, so] * eo
pdR_D_W[:, :, sv, sv] -= pdR_D_r[:, :, sv, sv] * ev
pdR_D_W[:, :, so, so] -= np.einsum("Atki, kj -> Atij", F_1R[:, :, so, so], D_r[so, so])
pdR_D_W[:, :, sv, sv] -= np.einsum("Atca, cb -> Atab", F_1R[:, :, sv, sv], D_r[sv, sv])
pdR_D_W[:, :, sv, so] -= np.einsum("Atki, ak -> Atai", F_1R[:, :, so, so], D_r[sv, so])

# WIII's code is the same as before, but result value is different compared with that before
pdR_D_W[:, :, so, so] -= 0.5 * hfh.Ax0_Core(so, so, sa, sa)(pdR_D_r)
pdR_D_W[:, :, so, so] -= 0.5 * hfh.Ax1_Core(so, so, sa, sa)(np.array([[D_r]]))[:, 0, :, 0]
pdR_D_W[:, :, so, so] -= 0.5 * np.einsum("Atmi, mj -> Atij", U_1R[:, :, :, so], hfh.Ax0_Core(sa, so, sa, sa)(D_r))
pdR_D_W[:, :, so, so] -= 0.5 * np.einsum("Atmj, im -> Atij", U_1R[:, :, :, so], hfh.Ax0_Core(so, sa, sa, sa)(D_r))
pdR_D_W[:, :, so, so] -= 0.5 * Ax0_Core(so, so, sa, sa)(np.einsum("Atmp, pq -> Atmq", U_1R, D_r))
pdR_D_W[:, :, so, so] -= 0.5 * Ax0_Core(so, so, sa, sa)(np.einsum("Atmq, pq -> Atmp", U_1R, D_r))

# part of contribution of original contrib4
hess_mp2_contrib3_4 += np.einsum("Bspq, Atpq -> ABts", pdR_D_W, S_1_mo)

In [27]:
# finally, add RHS * U_1_vo
hess_mp2_contrib3_4 += np.einsum("Atai, Bsai -> ABts", U_1_vo, RHS)

In [28]:
hess_mp2 = (
    + hess_mp2_contrib1
    + hess_mp2_contrib2
    + hess_mp2_contrib3_4
    + hess_mp2_contrib5_6
)

In [29]:
np.allclose(hess_mp2, hess_mp2.swapaxes(0, 1).swapaxes(2, 3))

True

In [30]:
print((hess_mp2 - hess_mp2_ref).max())
print((hess_mp2 - hess_mp2_ref).min())

5.8013600867656834e-08
-6.048166338590288e-08
